In [60]:
# import pandas as pd
# gxsapple=pd.read_csv("gxs-apple-app-reviews.csv", index_col=0)
# gxsplaystore=pd.read_csv("gxs_playstore2.csv", index_col=0)

In [72]:
# gxsapple['title_review'] = gxsapple['title'] + ' : ' + gxsapple['review']

# gxsapple['thumbsUp']= 0

# gxsapple['developerResponse'] = gxsapple['developerResponse'].apply(lambda x: x.get('body') if isinstance(x, dict) else x)
# gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace(r"\{'id': \d+, 'body':", "", regex=True)
# gxsapple['developerResponse']=gxsapple['developerResponse'].str.rstrip("}")
# gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace('"', '', regex=False)
# gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace("'", '', regex=False)

# gxsapple_dropped = gxsapple.drop(columns=['title', 'review', 'userName', 'isEdited'])

# gxsplaystore_dropped=gxsplaystore.drop(columns=['reviewId','userName','userImage','reviewCreatedVersion','repliedAt','appVersion'])
# neworder=['replyContent', 'score', 'content', 'thumbsUpCount', 'at']
# gxsplaystore_dropped=gxsplaystore_dropped[neworder]

In [73]:
# gxsapple_dropped

,date,developerResponse,rating,title_review,thumbsUp
0,2024-03-06 08:36:37,Hi Eugene! Were thrilled to know that you lik...,4,App can have more features! : Great banking ap...,0
1,2022-11-09 03:13:16,Hi Dfdsa132! Were delighted that you love our...,5,"A big bang! : A bank like no other, no bank ha...",0
2,2023-09-09 08:33:43,Hi there! We’re sorry to make you feel that w...,1,Unreasonable drop in interest rate : Notice th...,0
3,2023-06-27 04:01:39,"Hi Kenny, thank you for alerting us to this i...",2,Unable to transfer funds back to myself : Send...,0
4,2022-10-02 04:17:49,"Hi Mephosis, we are happy that you like our a...",5,Potentially the best UI/UX among all the new d...,0
...,...,...,...,...,...
127,2023-09-29 13:18:27,Thank you very much for your kind support! We...,5,Wow : Wow,0
128,2022-09-16 18:40:11,Hey Jasper! We are happy to know that you lik...,5,🚀🚀🚀🚀🚀 : 🚀🚀🚀🚀🚀,0
129,2023-07-24 03:31:10,Hi there. We apologize for the inconvenience ...,1,Scan and pay amount wrong : I was trying to ma...,0
130,2023-07-22 01:33:39,Hi there! Were sorry you feel that way. Fret ...,1,Bug ! Bug ! Bug ! : 安全漏洞！下载要小心！即使用SingPass 开户也...,0


In [74]:
# gxsplaystore_dropped=gxsplaystore_dropped.rename(columns={
#     'content': 'content',
#     'thumbsUpCount': 'thumbsUpCount',
#     'replyContent': 'replyContent',
#     'score': 'score',
#     'at':'date'
# })

# gxsapple_dropped_renamed = gxsapple_dropped.rename(columns={
#     'title_review': 'content',
#     'thumbsUp': 'thumbsUpCount',
#     'developerResponse': 'replyContent',
#     'rating': 'score',
#     'date':'date'
# })

In [75]:
# combined_reviews = pd.concat([gxsplaystore_dropped, gxsapple_dropped_renamed], axis=0, ignore_index=True)

In [76]:
# if combined_reviews['replyContent'].apply(lambda x: isinstance(x, dict)).any():
#     combined_reviews['replyContent'] = combined_reviews['replyContent'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

In [77]:
# combined_reviews.to_csv("combined_reviews.csv")

In [28]:
from keybert import KeyBERT
from transformers import pipeline
import pandas as pd
import seaborn as sns
import pandas as pd
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline

/Users/leeeda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/leeeda/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
dataset = pd.read_csv('combined_reviews.csv', index_col=0)

In [79]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date
0,"Hey Jiaxing! Hope you love the new ""Favourites...",5,"Gxs is simple and easy to use, with a saving a...",13,2024-03-22 09:33:40
1,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,2024-03-16 20:03:04
2,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,2024-03-16 15:18:24
3,"Hi Xiangqing, we'd love to look into this for ...",1,cant logging using finger print . previously s...,0,2024-03-10 23:51:02
4,"Hi Matt, thanks for reaching out! If you are r...",1,For 1st time registration signup by singpass a...,0,2024-03-10 22:02:20


In [33]:
sa_model = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank'
sentiment_pipeline = pipeline("sentiment-analysis", model=sa_model)

In [34]:
ke_model = KeyBERT()

In [35]:
ke_model.extract_keywords(dataset.content[50])

[('rating', 0.5417),
 ('ratings', 0.4779),
 ('rate', 0.3851),
 ('app', 0.3224),
 ('unfair', 0.1467)]

In [38]:
baseline_model = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(baseline_model)

save_path = "./gpt_model_causallm"
finetuned_model = AutoModelForCausalLM.from_pretrained(save_path)

generator = pipeline('text-generation', finetuned_model, tokenizer=tokenizer)
inst = 'Below is a review for a banking app. Write a response that appropriately replies to the review from the perspective of the bank.'

In [81]:
sentiment = []
keywords = []
replies = []
for index, row in dataset[199:].iterrows():
    review = row['content']
    s = sentiment_pipeline(review)[0]['label']
    sentiment.append(s)
    k = ke_model.extract_keywords(review)
    k = [i[0] for i in k]
    keywords.append(k)
    prompt = f'{inst} ### Review: {review} ### Reply:'
    reply = generator(prompt, max_length=len(review)+50)[0]['generated_text'][152+len(review):]
    replies.append(reply)

dataset['sentiment'] = sentiment
dataset['keywords'] = keywords
dataset['replies'] = replies

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [88]:
dataset = dataset.rename(columns={'replies':'generatedReply'})

In [89]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date,sentiment,keywords,generatedReply
0,"Hey Jiaxing! Hope you love the new ""Favourites...",5,"Gxs is simple and easy to use, with a saving a...",13,2024-03-22 09:33:40,POSITIVE,"[gxs, funds, debit, investment, card]",Hi Matt! Welcome back to the review from the ...
1,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,2024-03-16 20:03:04,NEGATIVE,"[finger, identification, ask, given]",Great! I appreciate that your prompt attentio...
2,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,2024-03-16 15:18:24,NEGATIVE,"[apps, app, whitelist, phone, using]",Please help us understand if you see any of t...
3,"Hi Xiangqing, we'd love to look into this for ...",1,cant logging using finger print . previously s...,0,2024-03-10 23:51:02,NEGATIVE,"[logging, finger, print, update, using]",uhhh the problem is i already have rooted the...
4,"Hi Matt, thanks for reaching out! If you are r...",1,For 1st time registration signup by singpass a...,0,2024-03-10 22:02:20,NEGATIVE,"[singpass, signup, registration, denied, appli...","Hi L.L., We are sorry for this inconvenience...."


In [90]:
dataset.to_csv('sentiment_data.csv')

In [92]:
gxs_reviews = dataset[['content', 'thumbsUpCount']]

In [93]:
def summarise_sentiment(reviews): #havent test this fn

  '''
  Input:
  - reviews: dataset containing at least the following 2 columns ['content', 'thumbsUpCount']
  Output:
  - net sentiment score: numeric, between -100 and +100
  - neg_keywords: dictionary containing negative keywords, sorted descending
  - pos_keywords: dictionary containing positive keywords, sorted descending
  '''

  model = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank' # maybe need to streamline this && change to trained model
  sentiment_pipeline = pipeline("sentiment-analysis", model=model)
  ns_score = 0

  kw_model = KeyBERT() # check if this is the best model to use
  neg_keywords = {}
  pos_keywords = {}

  for row in range(len(reviews)):
    pred_sentiment = sentiment_pipeline(reviews.content[row])[0]['label']
    keywords = kw_model.extract_keywords(reviews.content[row])
    if pred_sentiment == 'POSITIVE':
      pred_sentiment = 1
      for word in keywords:
        if word in pos_keywords:
          pos_keywords[word] += 1
        else:
          pos_keywords[word] = 1
    else:
      pred_sentiment = -1
      for word in keywords:
        if word in neg_keywords:
          neg_keywords[word] += 1
        else:
          neg_keywords[word] = 1
    ns_score += pred_sentiment * (reviews.thumbsUpCount[row]+1) # need to vectorise for faster runtime

  total = sum(reviews.thumbsUpCount) + len(reviews)
  ns_score = ns_score/total*100

  pos_keywords = dict(sorted(pos_keywords.items(), key=lambda x: x[1], reverse = True))
  neg_keywords = dict(sorted(neg_keywords.items(), key=lambda x: x[1], reverse = True))

  return ns_score, pos_keywords, neg_keywords

In [94]:
summary = summarise_sentiment(gxs_reviews)

In [95]:
summary

(-50.877192982456144,
 {('good', 0.794): 2,
  ('gxs', 0.5408): 1,
  ('funds', 0.3154): 1,
  ('debit', 0.3088): 1,
  ('investment', 0.3084): 1,
  ('card', 0.2983): 1,
  ('pockets', 0.5949): 1,
  ('allocation', 0.3315): 1,
  ('multiple', 0.3242): 1,
  ('track', 0.2953): 1,
  ('numbers', 0.2623): 1,
  ('loans', 0.5687): 1,
  ('bank', 0.534): 1,
  ('repayments', 0.5072): 1,
  ('transfers', 0.4789): 1,
  ('fast', 0.3366): 1,
  ('ok', 0.4395): 1,
  ('good', 0.4314): 1,
  ('bank', 0.4198): 1,
  ('savings', 0.3572): 1,
  ('annum', 0.2845): 1,
  ('payment', 0.2533): 1,
  ('saving', 0.2395): 1,
  ('savings', 0.5343): 1,
  ('pockets', 0.3881): 1,
  ('app', 0.3704): 1,
  ('park', 0.2451): 1,
  ('economic', 0.2424): 1,
  ('password', 0.4833): 1,
  ('apps', 0.4585): 1,
  ('load', 0.3312): 1,
  ('phone', 0.2852): 1,
  ('set', 0.232): 1,
  ('app', 0.6837): 1,
  ('good', 0.3168): 1,
  ('gxs', 0.5238): 1,
  ('bank', 0.4397): 1,
  ('pay', 0.3048): 1,
  ('google', 0.2317): 1,
  ('available', 0.2302): 1,
 

In [112]:
summary_df = pd.DataFrame(summary, index=['netSentiment', 'positiveKeywords', 'negativeKeywords'])

In [114]:
summary_df

,0
netSentiment,-50.877193
positiveKeywords,"{('good', 0.794): 2, ('gxs', 0.5408): 1, ('fun..."
negativeKeywords,"{('app', 0.419): 2, ('unable', 0.3002): 2, ('a..."


In [115]:
summary_df.to_csv('summary.csv')